In [152]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [147]:
## load the trained model , scaler pickle,onehot
model = load_model('deep_learning_model/model.h5')

## load the encoder and scaler
with open('pickle_files/label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('pickle_files/onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('pickle_files/scaler.pkl','rb') as file:
    scaler = pickle.load(file)


In [148]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [149]:
geo_encoded = onehot_encoder_geo.transform(
    [[input_data['Geography']]]
)
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [150]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [153]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [154]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [155]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.51678002,  0.916224  ,  0.09079683, -0.68850902, -0.27193243,
         0.80725845,  0.64737281,  0.97286124, -0.86915921,  1.00601962,
        -0.57802401, -0.58129653]])

In [144]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step


array([[0.05013325]], dtype=float32)

In [145]:
prediction_proba = prediction[0][0]

In [146]:
prediction_proba

np.float32(0.050133247)

In [151]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
